In [1]:
import json, folium
import numpy as np
import pandas as pd

# 1. 자치구별 공원면적을 단계구분도 

In [12]:
pop = pd.read_csv('test_data/서울시 구별 CCTV 인구 현황.csv', index_col='구별')
pop

,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인 비율,고령자 비율
구별,,,,,,,,
종로구,1835,28.77,152212,141060,11152,28265,7.33,18.57
중구,2584,34.87,131390,120963,10427,25353,7.94,19.30
용산구,2970,50.81,232482,217756,14726,39478,6.33,16.98
성동구,4040,26.83,287240,280240,7000,48238,2.44,16.79
광진구,3480,34.51,350925,336801,14124,54854,4.02,15.63
동대문구,2750,28.18,354884,337574,17310,65154,4.88,18.36
중랑구,4193,42.38,389928,385003,4925,76116,1.26,19.52
성북구,4853,20.56,441855,430100,11755,78427,2.66,17.75
강북구,3321,50.11,296934,292762,4172,67056,1.41,22.58


In [ ]:
park = pd.read_csv('test_data/서울공원요약.csv')
park                   

In [6]:
park['자치구'] = park.주소.apply(lambda x: x.split()[1])
park.자치구.unique()

array(['중구', '강동구', '과천시', '성동구', '마포구', '종로구', '영등포구', '강서구', '동대문구',
       '동작구', '광진구', '용산구', '서대문구', '서초구', '관악구', '금천구', '성북구', '구로구',
       '양천구', '강남구', '송파구', '도봉구', '강북구', '은평구', '중랑구', '노원구', '공릉2동'],
      dtype=object)

In [8]:
# 잘못된 데이터중 공릉2동은 노원구로 변경
park.자치구.replace('공릉2동','노원구', inplace=True)
park.자치구.unique()

array(['중구', '강동구', '과천시', '성동구', '마포구', '종로구', '영등포구', '강서구', '동대문구',
       '동작구', '광진구', '용산구', '서대문구', '서초구', '관악구', '금천구', '성북구', '구로구',
       '양천구', '강남구', '송파구', '도봉구', '강북구', '은평구', '중랑구', '노원구'],
      dtype=object)

In [9]:
park_df = park.pivot_table('면적', '자치구', aggfunc='sum')
park_df.head()

,면적
자치구,
강남구,3626398
강동구,1526387
강북구,2047054
강서구,2194152
과천시,9132690


In [11]:
park_df.drop(['과천시'], inplace=True)
park_df.head()

,면적
자치구,
강남구,3626398
강동구,1526387
강북구,2047054
강서구,2194152
관악구,10431453


In [13]:
df = park_df.merge(pop, left_index=True, right_index=True)
df.head()

,면적,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인 비율,고령자 비율
강남구,3626398,7243,26.30,537817,532798,5019,83097,0.93,15.45
강동구,1526387,3190,42.79,464027,459982,4045,79706,0.87,17.18
강북구,2047054,3321,50.11,296934,292762,4172,67056,1.41,22.58
강서구,2194152,3312,42.27,573711,568287,5424,98659,0.95,17.20
관악구,10431453,5239,20.77,502628,487815,14813,83420,2.95,16.60


In [ ]:
geo_data = json.load(open('test_data/seoul_geo_simple.json', encoding='utf-8'))
geo_data                     

In [14]:
df['인당 면적'] = (df.면적 / df.인구수).round(2)
df = df[['면적','인구수','인당 면적']]
df.head()

,면적,인구수,인당 면적
강남구,3626398,537817,6.74
강동구,1526387,464027,3.29
강북구,2047054,296934,6.89
강서구,2194152,573711,3.82
관악구,10431453,502628,20.75


In [15]:
# 자치구 이름쓰기
import numpy as np
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:, 1]), np.mean(geo[:,0])]
    return gu_dict

In [18]:
gu_dict = get_text_location(geo_data)

# 2.자치구별 공원면적을 단계구분도 

In [19]:
map = folium.Map([37.55, 126.98], zoom_start=10)
folium.Choropleth(
    geo_data=geo_data,
    data=df.면적,       # 단계구분도로 보여줄 데이터
    columns=[df.index, df.면적],    # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',      # Colormap
    key_on='feature.id'     # 지도에서 조인할 항목
).add_to(map)
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 공원 면적</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

In [21]:
map = folium.Map([37.55, 126.98], zoom_start=10)
folium.Choropleth(
    geo_data=geo_data,
    data=df['인당 면적'],       # 단계구분도로 보여줄 데이터
    columns=[df.index, df['인당 면적']],    # 데이터프레임에서 추출할 항목
    fill_color='GnBu',      # Colormap
    key_on='feature.id'     # 지도에서 조인할 항목
).add_to(map)
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 인당 공원면적</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map